# Pycipher

In [1]:
from pycipher import ADFGVX
adfgvx = ADFGVX(key='PH0QG64MEA1YL2NOFDXKR3CVS5ZW7BJ9UTI8', keyword='GERMAN')
adfgvx.encipher("Hello world!")
adfgvx.decipher(adfgvx.encipher("Hello world!"))

'HELLOWORLD'

In [2]:
import functools
import numpy as np
import operator


## Playfair Cipher


In [3]:
def get_index(m: list, c):
    i = m.index(c)
    return i // rows, i % cols

def play_fair_encode(plaintext: str, alphabet: list):
    # Substitute 'J' for 'I'
    message = ''.join([m for m in plaintext.upper().replace("J", "I") if m in alphabet])
    # If any character pair is the same character, insert the padding 'X'
    message = ''.join(functools.reduce(
                        operator.iconcat, 
                        [[c1, 'X', c2] if c1 == c2 else [c1, c2] for c1, c2 in zip(message[0::2], (message + " ")[1::2])], 
                        [])).replace(' ', '')
    # Ensure we have a even length message by padding last character with X
    message += "X" if (len(message) % 2 == 1) else ""
    
    ciphertext = ""
    for c1, c2 in zip(message[0::2], message[1::2]):
        r1, c1 = get_index(alphabet, c1)
        r2, c2 = get_index(alphabet, c2)
        if r1 == r2:
            ciphertext += alphabet[r1 * cols + ((c1 + 1) % cols)]
            ciphertext += alphabet[r1 * cols + ((c2 + 1) % cols)]
        elif c1 == c2:
            ciphertext += alphabet[((r1 + 1) % rows) * cols + c1]
            ciphertext += alphabet[((r2 + 1) % rows) * cols + c1]
        else:
            ciphertext += alphabet[r1 * cols + c2]
            ciphertext += alphabet[r2 * cols + c1]            
    return ciphertext

def play_fair_decode(ciphertext: str, alphabet: list):
    plaintext = ""
    for c1, c2 in zip(ciphertext[0::2], ciphertext[1::2]):
        r1, c1 = get_index(alphabet, c1)
        r2, c2 = get_index(alphabet, c2)
        if r1 == r2:
            plaintext += alphabet[r1 * cols + ((c1 - 1) % cols)]
            plaintext += alphabet[r1 * cols + ((c2 - 1) % cols)]
        elif c1 == c2:
            plaintext += alphabet[((r1 - 1) % rows) * cols + c1]
            plaintext += alphabet[((r2 - 1) % rows) * cols + c1]
        else:
            plaintext += alphabet[r1 * cols + c2]
            plaintext += alphabet[r2 * cols + c1]            
    return plaintext

In [4]:
keyword = "STUAR"
alphabet = list(keyword + ''.join([c for c in "ABCDEFGHIKLMNOPQRSTUVWXYZ" if c not in keyword]))

rows = 5; cols = 5;
assert len(alphabet) == rows * cols and len(sorted(set(alphabet))) == len(alphabet)
matrix_5x5 = np.matrix(np.char.array([list(alphabet[i * cols:(i + 1) * cols]) for i in range(rows)]))
print(matrix_5x5)


plain_text = "Friends romans countrymen lend me your ears!"
cipher_text = play_fair_encode(plain_text, alphabet)

print(plain_text)
print(cipher_text)
print(play_fair_decode(cipher_text, alphabet))

[['S' 'T' 'U' 'A' 'R']
 ['B' 'C' 'D' 'E' 'F']
 ['G' 'H' 'I' 'K' 'L']
 ['M' 'N' 'O' 'P' 'Q']
 ['V' 'W' 'X' 'Y' 'Z']]
Friends romans countrymen lend me your ears!
LFKDOCTSPNTPTBXDWCAZPBQHCPBOKAXDAFRSUV
FRIENDSROMANSCOUNTRYMENLENDMEYOUREARSX


## Hill Cipher


In [5]:
def hill_transform(m, message):
    n = m.shape[0] # Assuming m is a square matrix
    # Pad message with 23
    pad_length = n - len(message) % n
    if pad_length < n:
        message.extend([23] * pad_length)
    
    # Perform the transformation
    result = []
    for i in range(0, len(message), n):
        chunk = np.array(message[i:i+n])
        transformed = np.dot(m, chunk)
        result.extend(transformed)
    
    # Apply modulo 26 to the result
    result = [x % 26 for x in result]
    return result

### Check key and the inverse

$
Encipher: E = \begin{pmatrix}
6 & 24 & 1 \\
13 & 16 & 10 \\
20 & 17 & 15
\end{pmatrix}
Decipher: D = E^{-1} = \begin{pmatrix}
8 & 5 & 10 \\
21 & 8 & 21 \\
21 & 12 & 8
\end{pmatrix}
$


In [6]:
def inverse_modulo_n(E, n):
    detE = int(round(np.linalg.det(E)))
    detE_inv_mod = pow(detE, -1, n)
    adjE = np.linalg.inv(E) * detE
    invE_mod = (detE_inv_mod * adjE) % n
    return np.rint(invE_mod).astype(int)

In [7]:
E = np.array([[6, 24, 1],
              [13, 16, 10],
              [20, 17, 15]])
D = inverse_modulo_n(E, 26)
print(D)

[[ 8  5 10]
 [21  8 21]
 [21 12  8]]


In [10]:
def string_to_integers(plain_text):
    # Convert to uppercase and filter out non-[A..Z] characters
    filtered_text = ''.join(filter(str.isalpha, plain_text.upper()))
    # Convert characters to integers (0-25)
    int_list = [ord(char) - ord('A') for char in filtered_text]
    return int_list
    
def integers_to_string(int_list):
    # Convert integers (0-25) to characters
    char_list = [chr(i + ord('A')) for i in int_list]
    # Join the list of characters into a string
    result_string = ''.join(char_list)
    return result_string

In [11]:
# Example usage
plain_text = 'Hello, Meet me at the toga party!'
print(string_to_integers(plain_text))
print(integers_to_string(string_to_integers(plain_text)))

[7, 4, 11, 11, 14, 12, 4, 4, 19, 12, 4, 0, 19, 19, 7, 4, 19, 14, 6, 0, 15, 0, 17, 19, 24]
HELLOMEETMEATTHETOGAPARTY


In [ ]:
print(plain_text)
print(convert_string_to_integers(plain_text))
print(hill_transform(E, convert_string_to_integers(plain_text)))
cipher_text = convert_string_to_integers(hill_transform(E, convert_string_to_integers(plain_text)))
print(cipher_text)
print(hill_transform(D, convert_string_to_integers(cipher_text)))
print(convert_string_to_integers(hill_transform(D, convert_string_to_integers(cipher_text))))

Hello, Meet me at the toga party!
[7, 4, 11, 11, 14, 12, 4, 4, 19, 12, 4, 0, 19, 19, 7, 4, 19, 14, 6, 0, 15, 0, 17, 19, 24]
[19, 5, 9, 24, 19, 14, 9, 20, 17, 12, 12, 22, 5, 23, 2, 0, 2, 15, 25, 20, 7, 11, 20, 2, 17, 0, 20]
TFJYTOJURMMWFXCACPZUHLUCRAU
[7, 4, 11, 11, 14, 12, 4, 4, 19, 12, 4, 0, 19, 19, 7, 4, 19, 14, 6, 0, 15, 0, 17, 19, 24, 23, 23]
HELLOMEETMEATTHETOGAPARTYXX
